　　训练神经网络是非常复杂的，在训练时每层的输入分布在改变，因为之前图层的参数在变化。这导致需要我们降低学习速率和谨慎初始化参数，从而降低了速率，并且使用非线性模型进行训练是很困难的。我们将这种想象称为 'internal covariate
shift'，通过标准化每层的输入可以解决该问题。<br />
　　我们的方法是将标准化作为模型框架的一部分，并对训练时每个mini-batch进行标准化。其好处是允许我们使用更高的学习速率和不用那么谨慎初始化参数。它也有正则化的作用，在某些情况不需要dropout。

### 介绍
　　SGD用于优化网络的参数 $\Theta $，即最小化损失：
$$\Theta = \underset{\Theta }{argmin} \frac{1}{N} \sum_{i=1}^{N} l (x_{i}, \Theta)$$

　　其中 $x_{1} ... x_{N}$ 是训练集。使用SGD，mini-batch大小为m，通常mini-batch用于近似损失函数的梯度：
$$\frac{1}{m} \frac{\partial l (x_{i}, \Theta )}{\partial \Theta }$$

　　相对于一次使用一个样本，使用mini-batch有几个优势：使用mini-batch求得的损失梯度使训练集梯度的估计， 其质量随着batch大小的增加而提高；对batch的计算比m个单一样本的计算更有效率，因为现代计算平台提供平行计算。
　　SGD是简单且高效的方法，但它需要小心调整模型超参数，尤其是学习速率，还有模型参数的初始值。训练过程是非常复杂的，因为每层的输入会受到前一层参数影响，所以随着网络越来越深，网络参数小小的改变都会放大。
　　因为每层输入的分布改变，导致这些层需要一直适应新的分布。当学习系统的输入分布改变时，会经历 covariate shift。这通常是通过 domain adaptation 来处理。covariate shift 概率可以扩展到整个学习系统之外，适用其部分，比如子网络或一层神经层。考虑一个网络的计算如下：
$$l = F_{2} (F_{1} (u, \Theta _{1}), \Theta _{2})$$

　　其中 $F_{1}$ 和 $F_{2}$ 是任意转换（激活函数），学习更新参数 $\Theta _{1}, \Theta _{2}$ 来最小化损失函数 $l$。 如果输入 $x=F_{1} (u, \Theta _{1})$ 放入子网络来训练参数 $\Theta _{2}$ :
$$l = F_{2} (x, \Theta _{2})$$
　　举例，梯度下降：
$$\Theta _{2} \leftarrow \Theta _{2} - \frac{\alpha }{m} \sum_{i=1}^{m} \frac{\partial  F_{2}(x_{i}, \Theta _{2})}{\partial  \Theta _{2}}$$

　　对于batch大小为 $m$ 和学习速率为 $\alpha $，这是个输入为X的独立网络 $F_{2}$。因此，让训练更有效率的输入分布的性质，比如训练集和测试集拥有相同的分布，还有训练的子网络也是。输入X的分布随时间保持固定是有利的。然后，参数 $\Theta _{2}$ 不需要随着输入X分布的改变而调整。<br />
　　固定子网络的输入的分布也会对子网络外的其它层产生积极影响。考虑一个sigmoid激活函数的层 $z=g(Wu + b)$，其中 u 是该层的输入，权重 w 和偏差 b 是该层要学习的参数，激活函数为 $g(x) = \frac {1}{1+exp(-x)}$。随着 $|x|$ 变大， $g'(x)$ 会趋于0。这意味着对于 $x=Wu+b$ 的所有维度，除了那些绝对值小的维度，u 的梯度将会消失，模型训练会变慢。但是，因为 x 是受到权重 w，偏差 b 和该层以下所有层的参数的影响。在训练期间对这些参数的改变可能会将 x 的许多维度移动到非线性的饱和状态，减慢收敛。这种影响随着网络深度的增加而增加。<br />
　　在实践中，饱和问题和导致梯度消失的问题通常可使用ReLU，谨慎初始化和小的学习速率解决。但是，我们需要确认非线性的输入分布随着网络训练须更加稳定，那么优化器将不太可能陷入饱和状态，并且训练会加速。

　　我们将在训练时深度网络内部节点分布的变化叫做 Internal Covariate Shift，消除它将会使训练更快。我们提出了一种新的机制，叫做 Batch Normalization（以下简称 BN ），用于减少 Internal Covariate Shift，这样做极大地加速了深层神经网络的训练。它通过一个标准化的步骤来完成，固定层的输入的均值和方差。BN 对网络的梯度也有积极影响，可减少对参数缩放的梯度的依赖或者初始值的依赖。这允许我们使用更大的学习速率。更进一步，BN 对网络有正则作用，可减少对dropout的需要。最后，BN 可以使用非线性激活函数，而不会使网络陷入饱和问题或梯度消失问题。

### 减少 Internal Covariate Shift

　　定义 Internal Covariate Shift 为由于训练期间网络参数的改变导致的网络激活分布的变化。为了提高训练速度，我们需要减少 Internal Covariate Shift 。通过在训练过程中固定层输入 x 的分布，提高训练的速度。如果对输入进行 whitened，比如线性转换为均值为0，方差为1并去相关，能加快网络的训练收敛速度。由于每层的输入都是前面层产生的，通过对每层的输入进行 whitening，我们可以固定输入的分布，以移除  Internal Covariate Shift 的影响。<br />
　　考虑对训练步骤或某些节点的激活输入进行whitening，通过直接修改网络结构或改变依赖网络激活值的优化方法的参数。但是，如果这些修改方法是穿插在优化步骤中，则梯度下降